## Simple Threat Analysis with Pandas

This notebook demonstrates a simple way to identify known bad actors within egress transaction generated from wire data traffic. Python Pandas is used to compare the IP addresses of servers exfiltrating from our 'friendly" set of IP addresses against known threats. 
 
This example uses IP addresses that have been reported within the last 48 hours as having run attacks on the service Mail, Postfix at the Blocklist website: https://lists.blocklist.de/lists/mail.txt, downloaded Jan 9

In [41]:
import pandas as pd
import os
import numpy

This test 'friendly' data is 10,000 records of egress transaction.
Read the data in pandas. 

In [42]:
egress=pd.read_csv("Traffic.csv", encoding='latin-1')

In [43]:
egress.head(5)

,Time,Record Type,Source,Destination,Source Location,Dest Location,Environment,Dest Country,Protocol,Client Address,Client Bytes,Server Address,Server Bytes,Latency,Process Time
0,55:39.4,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,telnet,172.45.8.6,3,172.45.8.56,36,NaN,NaN
1,55:38.5,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,1,NaN,17.626
2,55:38.5,Flow Audit,Santa Clara Campus (Users),External,Santa Clara Office,California,EGRESS,United States,SSL:443,192.168.0.104,173,173.194.33.144,133,NaN,NaN
3,55:38.5,Flow Audit,Santa Clara Campus (Users),External,Santa Clara Office,California,EGRESS,United States,SSL:443,192.168.0.104,600,173.194.33.144,"1,406",NaN,0.747
4,55:38.3,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,1,NaN,64.652


We can quickly identify the countries and volume of traffice associated with the egress servers and dig deeper into transactions linked to a country of concern, for example 'France.

In [44]:
servers=egress['Dest Country']

In [45]:
servers.value_counts()

United States     9580
Norway             291
Ireland             51
Japan               22
France              13
Netherlands         10
Singapore            5
Poland               1
United Kingdom       1
Name: Dest Country, dtype: int64

In [46]:
France=(egress.loc[egress['Dest Country'].isin(['France'])])

In [47]:
France.head(2)

,Time,Record Type,Source,Destination,Source Location,Dest Location,Environment,Dest Country,Protocol,Client Address,Client Bytes,Server Address,Server Bytes,Latency,Process Time
1784,50:13.4,Flow Audit,Santa Clara Campus (Users),External,Santa Clara Office,Ì_le-de-France,EGRESS,France,SSL:443,192.168.0.99,126,74.121.138.36,258,NaN,57.071
1787,50:13.3,Flow Audit,Santa Clara Campus (Users),External,Santa Clara Office,Ì_le-de-France,EGRESS,France,SSL:443,192.168.0.99,517,74.121.138.36,"2,916",60.99,76.425


We can see that there are 13 transactions, all from the Santa Clara Campus, the most frequent destinationis Ale-de-France and 188.165.39.118 is the most common server address.	

In [48]:
France.describe()

,Time,Record Type,Source,Destination,Source Location,Dest Location,Environment,Dest Country,Protocol,Client Address,Client Bytes,Server Address,Server Bytes,Latency,Process Time
count,13,13,13,13,13,5,13,13,13,13,13,13,13,6,12
unique,9,1,1,1,1,1,1,1,1,1,8,4,9,6,12
top,40:26.4,Flow Audit,Santa Clara Campus (Users),External,Santa Clara Office,Ì_le-de-France,EGRESS,France,SSL:443,192.168.0.99,517,178.250.2.78,137,138.476,135.259
freq,2,13,13,13,13,5,13,13,13,13,3,4,2,1,1


In [49]:
server_ip=egress['Server Address']

We can also determen the frequency of each egress IP address in the test data set.

In [50]:
server_ip.describe()

count            10000
unique             691
top       52.70.108.65
freq               225
Name: Server Address, dtype: object

We can easily identify the top 20 most frequent ip addresses.

In [51]:
server_ip.value_counts().head(20)

52.70.108.65       225
152.195.32.119     221
208.43.234.47      196
74.125.30.189      181
173.194.33.75      152
192.229.163.180    147
69.172.216.111     145
104.96.86.223      142
108.174.11.65      132
192.0.72.2         114
173.194.33.69      105
23.59.189.82       104
172.45.8.97        101
91.203.99.19        99
172.45.8.81         97
172.45.8.83         94
172.45.8.98         92
172.45.8.62         90
172.45.8.86         90
172.45.8.78         89
Name: Server Address, dtype: int64

For our purposes we will extract just the egress 'Server Address' column.

In [52]:
justip=egress['Server Address']

In [53]:
justip=justip.to_frame()

In [54]:
type(justip)

pandas.core.frame.DataFrame

In [55]:
justip.columns =['Server Address']


In [56]:
justip.tail(5)

,Server Address
9995,173.194.33.65
9996,173.194.33.94
9997,54.230.69.112
9998,54.230.69.112
9999,54.230.69.112


We now have a list of IP addresses for our egress transactions that we want to compare against a list of known 'badactor' sites, called 'blocklist'. Blocklist contains 20,135 addresses. 

In [57]:
blocklist = pd.read_csv('badips.csv', header=None, names=['Server Address'])

In [58]:
blocklist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20135 entries, 0 to 20134
Data columns (total 1 columns):
Server Address    20135 non-null object
dtypes: object(1)
memory usage: 157.4+ KB


In [59]:
blocklist.tail(5)

,Server Address
20130,188.143.235.21
20131,95.134.130.182
20132,95.143.192.159
20133,176.100.75.27
20134,176.221.42.32


We merge the two data sets and find that there are 0 overlapping records.
Congratulations! These transactions are not terminating at a known malicious site.

In [60]:
joined_ips=pd.merge(egress,blocklist, on='Server Address', how='inner')

In [61]:
joined_ips.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 15 columns):
Time               0 non-null object
Record Type        0 non-null object
Source             0 non-null object
Destination        0 non-null object
Source Location    0 non-null object
Dest Location      0 non-null object
Environment        0 non-null object
Dest Country       0 non-null object
Protocol           0 non-null object
Client Address     0 non-null object
Client Bytes       0 non-null object
Server Address     0 non-null object
Server Bytes       0 non-null object
Latency            0 non-null object
Process Time       0 non-null object
dtypes: object(15)
memory usage: 0.0+ bytes


In [62]:
joined_ips.tail(20)

,Time,Record Type,Source,Destination,Source Location,Dest Location,Environment,Dest Country,Protocol,Client Address,Client Bytes,Server Address,Server Bytes,Latency,Process Time


Lets add some fake data (like fake news but better!) into our blocklist to make sure the join is working.  I have taken 10 IP addresses from the 'egress' data frame and concatonated it to the original blocklist to create fakeblocklist.

In [63]:
fakeblocklist=pd.read_csv('fakebadservers.csv', names=['Server Address'])

In [64]:
fakeblocklist=pd.concat([blocklist, fakeblocklist], axis=0)

In [65]:
fakeblocklist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20145 entries, 0 to 9
Data columns (total 1 columns):
Server Address    20145 non-null object
dtypes: object(1)
memory usage: 314.8+ KB


In [66]:
fakeblocklist.head(3)

,Server Address
0,1.173.37.30
1,1.192.229.40
2,1.194.116.78


Merging egress and fakeblock list identifies 580 transactions associated with a known 'bad actor'. 

In [67]:
joined_ips2=pd.merge(egress,fakeblocklist, on='Server Address', how='inner')

In [68]:
joined_ips2.head(5)

,Time,Record Type,Source,Destination,Source Location,Dest Location,Environment,Dest Country,Protocol,Client Address,Client Bytes,Server Address,Server Bytes,Latency,Process Time
0,55:38.5,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,1,NaN,17.626
1,55:38.3,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,1,NaN,64.652
2,55:38.0,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,3,172.45.8.59,3,NaN,"548,711.66"
3,55:36.5,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,1,NaN,297.186
4,55:36.2,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,4,NaN,61.68


In [69]:
joined_ips2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 580 entries, 0 to 579
Data columns (total 15 columns):
Time               580 non-null object
Record Type        580 non-null object
Source             580 non-null object
Destination        580 non-null object
Source Location    580 non-null object
Dest Location      580 non-null object
Environment        580 non-null object
Dest Country       580 non-null object
Protocol           580 non-null object
Client Address     580 non-null object
Client Bytes       573 non-null object
Server Address     580 non-null object
Server Bytes       573 non-null object
Latency            193 non-null object
Process Time       539 non-null object
dtypes: object(15)
memory usage: 72.5+ KB


I think Pandas offers a lot more analytic capability than Excel or SQL.